In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
def read_data():
    basedir = './prepped_data/'
    
    main_data_df = pd.read_csv(basedir + 'main_data_ver_2.4.csv')
    print(1)
    coor_df = pd.read_csv(basedir + 'coor_data_ver_1.csv')
    print(2)
    land_prices_df = pd.read_csv(basedir + 'land_prices_ver_1.csv')
    print(3)
    land_specs_df = pd.read_csv(basedir + 'land_specs_ver_2.csv')
    print(4)
    land_plans_df = pd.read_csv(basedir + 'land_plans_ver_2.csv')
    print(5)
    
    return main_data_df, coor_df, land_prices_df, land_specs_df, land_plans_df

In [3]:
%%time
main_data_df, coor_df, land_prices_df, land_specs_df, land_plans_df = read_data()

1


c:\users\max6296\ml\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


2
3
4
5
Wall time: 18.1 s


In [4]:
print(main_data_df.shape)
main_data_df.head()

(4559866, 16)


,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,9,1988,25,10.950807,6.597308
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,12,1988,25,10.950807,6.597308
2,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,55000,0,817.479191,5,2013,2,1987,26,10.915088,6.706225
3,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,58250,0,865.784780,4,2013,2,1987,26,10.972499,6.763636
4,서울특별시 강남구 개포동 658-1,서울특별시 강남구 언주로 3-0,일반,1,아파트_매매,67.28,60000,0,891.795482,5,2013,5,1987,26,11.002100,6.793237


In [5]:
merge1_df = main_data_df.merge(land_prices_df, on=['지번주소', '년'])
print(merge1_df.shape)
merge1_df.head()

(4405464, 19)


,지번주소,도로명주소,대장구분명,대장구분코드,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed,특수지구분명,공시지가,표준지여부
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,9,1988,25,10.950807,6.597308,일반,9210000,N
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_매매,77.75,57000,0,733.118971,2,2013,12,1988,25,10.950807,6.597308,일반,9210000,N
2,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_전세,77.75,35000,0,450.160772,4,2013,1,1988,25,10.463103,6.109605,일반,9210000,N
3,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_전세,77.75,32000,0,411.575563,9,2013,3,1988,25,10.373491,6.019993,일반,9210000,N
4,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,일반,1,아파트_전세,77.75,37000,0,475.884244,1,2013,9,1988,25,10.518673,6.165175,일반,9210000,N


In [6]:
land_specs_df = land_specs_df[land_specs_df['대장구분코드'] == 1].reset_index(drop=True)

In [7]:
merge1_df.drop(columns=['대장구분명', '대장구분코드'], inplace=True)

In [8]:
land_specs_df.drop(columns=['Unnamed: 0', '대장구분코드'], inplace=True)

In [36]:
merge2_df = merge1_df.merge(land_specs_df, on=['지번주소', '년'])
print(merge2_df.shape)
merge2_df.head()

(4397723, 24)


,지번주소,도로명주소,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건축년도,건물나이,target_log_transformed,target/area_log_transformed,특수지구분명,공시지가,표준지여부,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,아파트_매매,77.75,57000,0,733.118971,2,2013,9,1988,25,10.950807,6.597308,일반,9210000,N,8,1168.3,15,0,2,3,3
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,아파트_매매,77.75,57000,0,733.118971,2,2013,12,1988,25,10.950807,6.597308,일반,9210000,N,8,1168.3,15,0,2,3,3
2,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,아파트_전세,77.75,35000,0,450.160772,4,2013,1,1988,25,10.463103,6.109605,일반,9210000,N,8,1168.3,15,0,2,3,3
3,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,아파트_전세,77.75,32000,0,411.575563,9,2013,3,1988,25,10.373491,6.019993,일반,9210000,N,8,1168.3,15,0,2,3,3
4,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,아파트_전세,77.75,37000,0,475.884244,1,2013,9,1988,25,10.518673,6.165175,일반,9210000,N,8,1168.3,15,0,2,3,3


In [37]:
basedir = './feature_maps/'
trade_type_map_df = pd.read_csv(basedir + 'trade_type_map.csv')
special_land_map_df = pd.read_csv(basedir + 'special_land_map.csv')
standard_land_map_df = pd.read_csv(basedir + 'standard_land_map.csv')

In [38]:
trade_type_map = trade_type_map_df.to_dict(orient='records')[0]
special_land_map = special_land_map_df.to_dict(orient='records')[0]
standard_land_map = standard_land_map_df.to_dict(orient='records')[0]

In [39]:
merge2_df['trade_type'] = merge2_df['trade_type'].map(trade_type_map)
merge2_df['특수지구분명'] = merge2_df['특수지구분명'].map(special_land_map)
merge2_df['표준지여부'] = merge2_df['표준지여부'].map(standard_land_map)

In [40]:
merge2_df.drop(columns=['건축년도'], inplace=True)
merge2_df.head()

,지번주소,도로명주소,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건물나이,target_log_transformed,target/area_log_transformed,특수지구분명,공시지가,표준지여부,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,9,25,10.950807,6.597308,1,9210000,0,8,1168.3,15,0,2,3,3
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,12,25,10.950807,6.597308,1,9210000,0,8,1168.3,15,0,2,3,3
2,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,35000,0,450.160772,4,2013,1,25,10.463103,6.109605,1,9210000,0,8,1168.3,15,0,2,3,3
3,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,32000,0,411.575563,9,2013,3,25,10.373491,6.019993,1,9210000,0,8,1168.3,15,0,2,3,3
4,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,37000,0,475.884244,1,2013,9,25,10.518673,6.165175,1,9210000,0,8,1168.3,15,0,2,3,3


In [41]:
merge2_df['year_linear'] = (merge2_df['년'] - merge2_df['년'].min()) + ((merge2_df['월']-1)/12)

In [42]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [43]:
merge2_df['month_sin'] = merge2_df['월'].apply(month_circular_sine_func)
merge2_df['month_cos'] = merge2_df['월'].apply(month_circular_cosine_func)

In [44]:
merge2_df = merge2_df[merge2_df['건물나이'] >= 0]
merge2_df.shape

(4395795, 26)

In [45]:
coor_df.head()

,지번주소,도로명주소,x좌표,y좌표
0,NaN,서울특별시 관악구 난곡로24가길 18-0,948851.369529,1.941318e+06
1,NaN,서울특별시 종로구 성균관로15길 33-0,955591.635372,1.954533e+06
2,NaN,서울특별시 성북구 인촌로7길 70-0,957563.072605,1.954475e+06
3,NaN,서울특별시 서초구 강남대로91길 5-0,957617.535913,1.945960e+06
4,NaN,서울특별시 서초구 바우뫼로11길 54-0,958121.727726,1.941641e+06


In [46]:
landnum_df = coor_df[['지번주소', 'x좌표', 'y좌표']].dropna()
roadname_df = coor_df[['도로명주소', 'x좌표', 'y좌표']].drop(landnum_df.index).reset_index(drop=True)
landnum_df.reset_index(drop=True, inplace=True)

In [47]:
merge3_df = merge2_df.merge(roadname_df, on=['도로명주소'], how='left')
print(merge3_df.shape)
merge3_df.head()

(4395795, 28)


,지번주소,도로명주소,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건물나이,target_log_transformed,target/area_log_transformed,특수지구분명,공시지가,표준지여부,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,year_linear,month_sin,month_cos,x좌표,y좌표
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,9,25,10.950807,6.597308,1,9210000,0,8,1168.3,15,0,2,3,3,7.666667,-0.866025,-0.500000,960471.903756,1.942550e+06
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,12,25,10.950807,6.597308,1,9210000,0,8,1168.3,15,0,2,3,3,7.916667,-0.500000,0.866025,960471.903756,1.942550e+06
2,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,35000,0,450.160772,4,2013,1,25,10.463103,6.109605,1,9210000,0,8,1168.3,15,0,2,3,3,7.000000,0.000000,1.000000,960471.903756,1.942550e+06
3,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,32000,0,411.575563,9,2013,3,25,10.373491,6.019993,1,9210000,0,8,1168.3,15,0,2,3,3,7.166667,0.866025,0.500000,960471.903756,1.942550e+06
4,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,37000,0,475.884244,1,2013,9,25,10.518673,6.165175,1,9210000,0,8,1168.3,15,0,2,3,3,7.666667,-0.866025,-0.500000,960471.903756,1.942550e+06


In [48]:
merge4_df = merge3_df.merge(landnum_df, on=['지번주소'], how='left')
print(merge4_df.shape)
merge4_df.head()

(4395795, 30)


,지번주소,도로명주소,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건물나이,target_log_transformed,target/area_log_transformed,특수지구분명,공시지가,표준지여부,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,year_linear,month_sin,month_cos,x좌표_x,y좌표_x,x좌표_y,y좌표_y
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,9,25,10.950807,6.597308,1,9210000,0,8,1168.3,15,0,2,3,3,7.666667,-0.866025,-0.500000,960471.903756,1.942550e+06,960471.903756,1.942550e+06
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,12,25,10.950807,6.597308,1,9210000,0,8,1168.3,15,0,2,3,3,7.916667,-0.500000,0.866025,960471.903756,1.942550e+06,960471.903756,1.942550e+06
2,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,35000,0,450.160772,4,2013,1,25,10.463103,6.109605,1,9210000,0,8,1168.3,15,0,2,3,3,7.000000,0.000000,1.000000,960471.903756,1.942550e+06,960471.903756,1.942550e+06
3,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,32000,0,411.575563,9,2013,3,25,10.373491,6.019993,1,9210000,0,8,1168.3,15,0,2,3,3,7.166667,0.866025,0.500000,960471.903756,1.942550e+06,960471.903756,1.942550e+06
4,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,37000,0,475.884244,1,2013,9,25,10.518673,6.165175,1,9210000,0,8,1168.3,15,0,2,3,3,7.666667,-0.866025,-0.500000,960471.903756,1.942550e+06,960471.903756,1.942550e+06


In [49]:
merge4_df['x좌표'] = merge4_df['x좌표_x'].fillna(merge4_df['x좌표_y'])
merge4_df['y좌표'] = merge4_df['y좌표_x'].fillna(merge4_df['y좌표_y'])
merge4_df.dropna(subset=['x좌표', 'y좌표'], inplace=True)

In [50]:
merge4_df.drop(columns=['x좌표_x', 'y좌표_x', 'x좌표_y', 'y좌표_y'], inplace=True)
merge4_df.head()

,지번주소,도로명주소,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건물나이,target_log_transformed,target/area_log_transformed,특수지구분명,공시지가,표준지여부,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,year_linear,month_sin,month_cos,x좌표,y좌표
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,9,25,10.950807,6.597308,1,9210000,0,8,1168.3,15,0,2,3,3,7.666667,-0.866025,-0.500000,960471.903756,1.942550e+06
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,12,25,10.950807,6.597308,1,9210000,0,8,1168.3,15,0,2,3,3,7.916667,-0.500000,0.866025,960471.903756,1.942550e+06
2,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,35000,0,450.160772,4,2013,1,25,10.463103,6.109605,1,9210000,0,8,1168.3,15,0,2,3,3,7.000000,0.000000,1.000000,960471.903756,1.942550e+06
3,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,32000,0,411.575563,9,2013,3,25,10.373491,6.019993,1,9210000,0,8,1168.3,15,0,2,3,3,7.166667,0.866025,0.500000,960471.903756,1.942550e+06
4,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,37000,0,475.884244,1,2013,9,25,10.518673,6.165175,1,9210000,0,8,1168.3,15,0,2,3,3,7.666667,-0.866025,-0.500000,960471.903756,1.942550e+06


In [51]:
merge4_df['trade_type'].unique()

array([0, 1, 2, 6, 7, 8, 4, 5, 3], dtype=int64)

In [52]:
trade_type_map_df.head()

,아파트_매매,아파트_전세,아파트_월세,다세대연립_매매,다세대연립_전세,다세대연립_월세,오피스텔_매매,오피스텔_전세,오피스텔_월세
0,0,1,2,3,4,5,6,7,8


In [53]:
merge4_df['공시지가'] = merge4_df['공시지가'] / 10000
merge4_df.rename(columns={'공시지가':'공시지가(만원)'}, inplace=True)
merge4_df['토지면적'] = merge4_df['토지면적'].apply(np.log)
merge4_df = merge4_df[(merge4_df['trade_type']!=2)&(merge4_df['trade_type']!=5)&(merge4_df['trade_type']!=8)]
merge4_df.shape

(3569918, 28)

In [54]:
x_y_min_max = pd.read_csv('./feature_maps/x_y_min_max.csv')
x_y_min_max.head()

,x_min,x_max,y_min,y_max
0,938046.302665,971817.352783,1.937377e+06,1.965841e+06


In [55]:
merge4_df['x_norm'] = (merge4_df['x좌표'] - x_y_min_max['x_min'][0]) - ((x_y_min_max['x_max'][0] - x_y_min_max['x_min'][0])/2)
merge4_df['y_norm'] = (merge4_df['y좌표'] - x_y_min_max['y_min'][0]) - ((x_y_min_max['y_max'][0] - x_y_min_max['y_min'][0])/2)

In [56]:
merge4_df.drop(columns=['x좌표', 'y좌표'], inplace=True)
merge4_df['x_2nd'] = merge4_df['x_norm'] ** 2
merge4_df['y_2nd'] = merge4_df['y_norm'] ** 2
merge4_df['x*y'] = merge4_df['x_norm']*merge4_df['y_norm']
merge4_df['x_sin'] = np.sin(merge4_df['x_norm'])
merge4_df['y_sin'] = np.sin(merge4_df['y_norm'])
merge4_df['x_cos'] = np.cos(merge4_df['x_norm'])
merge4_df['y_cos'] = np.cos(merge4_df['y_norm'])

In [57]:
merge4_df.head()

,지번주소,도로명주소,trade_type,전용면적(㎡),금액(만원),월세(만원),가격/면적,층,년,월,건물나이,target_log_transformed,target/area_log_transformed,특수지구분명,공시지가(만원),표준지여부,지목코드,토지면적,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,year_linear,month_sin,month_cos,x_norm,y_norm,x_2nd,y_2nd,x*y,x_sin,y_sin,x_cos,y_cos
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,9,25,10.950807,6.597308,1,921.0,0,8,7.063305,15,0,2,3,3,7.666667,-0.866025,-0.500000,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,0,733.118971,2,2013,12,25,10.950807,6.597308,1,921.0,0,8,7.063305,15,0,2,3,3,7.916667,-0.500000,0.866025,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138
2,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,35000,0,450.160772,4,2013,1,25,10.463103,6.109605,1,921.0,0,8,7.063305,15,0,2,3,3,7.000000,0.000000,1.000000,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138
3,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,32000,0,411.575563,9,2013,3,25,10.373491,6.019993,1,921.0,0,8,7.063305,15,0,2,3,3,7.166667,0.866025,0.500000,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138
4,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,37000,0,475.884244,1,2013,9,25,10.518673,6.165175,1,921.0,0,8,7.063305,15,0,2,3,3,7.666667,-0.866025,-0.500000,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138


In [58]:
merge4_df.drop(columns=['월세(만원)'], inplace=True)
merge4_df.rename(columns={'토지면적':'토지면적_log'}, inplace=True)

In [59]:
merge4_df.head()

,지번주소,도로명주소,trade_type,전용면적(㎡),금액(만원),가격/면적,층,년,월,건물나이,target_log_transformed,target/area_log_transformed,특수지구분명,공시지가(만원),표준지여부,지목코드,토지면적_log,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,year_linear,month_sin,month_cos,x_norm,y_norm,x_2nd,y_2nd,x*y,x_sin,y_sin,x_cos,y_cos
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,733.118971,2,2013,9,25,10.950807,6.597308,1,921.0,0,8,7.063305,15,0,2,3,3,7.666667,-0.866025,-0.500000,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,733.118971,2,2013,12,25,10.950807,6.597308,1,921.0,0,8,7.063305,15,0,2,3,3,7.916667,-0.500000,0.866025,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138
2,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,35000,450.160772,4,2013,1,25,10.463103,6.109605,1,921.0,0,8,7.063305,15,0,2,3,3,7.000000,0.000000,1.000000,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138
3,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,32000,411.575563,9,2013,3,25,10.373491,6.019993,1,921.0,0,8,7.063305,15,0,2,3,3,7.166667,0.866025,0.500000,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138
4,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,1,77.75,37000,475.884244,1,2013,9,25,10.518673,6.165175,1,921.0,0,8,7.063305,15,0,2,3,3,7.666667,-0.866025,-0.500000,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138


In [60]:
land_plans_df.head()

,지번주소,대장구분명,년,(한강)오염행위 제한지역,(한강)폐기물매립시설 설치제한지역,가로구역별 최고높이 제한지역,가스공급설비,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고도지구,고등학교,고속철도,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공원,공원마을지구(공원집단시설지구),공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항소음피해예상지역,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광장,교육환경보호구역,교통광장,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,근린공원,근린광장,근린상업지역,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타문화시설,기타보건위생시설,기타수도시설,기타시장시설,기타용도지역지구기타,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타주차장시설,기타철도시설,기타폐기물처리시설,노외주차장,녹지,대공방어협조구역,대공방어협조구역(위탁고도:54-236m),대공방어협조구역(위탁고도:77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,묘지공원,문화공원,문화시설,문화재,문화재보존영향 검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,박물관,방송통신시설,방수설비,방화지구,변전소(전원개발사업구역),변전시설,보전산지,보전임지,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사회복지시설,산업개발진흥지구,산업시설구역,상대보호구역,상수원보호구역,생산녹지지역,소공원,소로1류(폭 10M~12M),소로2류(폭 8M~10M),소로3류(폭 8M 미만),소하천,소하천구역,수도공급시설,수평표면구역,시·도 생태·경관보전지역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,아파트지구,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,완충녹지,용도구역기타,용도구역미분류,운동장,원추표면구역,원형보존지,유수지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차전용도로,자동차정류장,자연경관지구,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종 구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(전술항공:5km),제한보호구역(후방지역:500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합의료시설,주간선도로,주거용지,주거환경개선지구,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭 20M~25M),중로2류(폭 15M~20M),중로3류(폭 12M~15M),중심대광장,중심상업지역,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,통제보호구역(민통선이남:300m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,하수도,하수종말처리시설,하천,하천구역,학교,학교이적지,현상변경허가 대상구역,홍수관리구역
0,서울특별시 강남구 개포동 1163-4,토지대장,2006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,서울특별시 강남구 개포동 1163-4,토지대장,2007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,서울특별시 강남구 개포동 1163-4,토지대장,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,서울특별시 강남구 개포동 1163-4,

In [62]:
print(merge4_df.shape)

(3569918, 34)


In [61]:
merge5_df = merge4_df.merge(land_plans_df, on=['지번주소', '년'])
print(merge5_df.shape)
merge5_df.head()

(3550570, 305)


,지번주소,도로명주소,trade_type,전용면적(㎡),금액(만원),가격/면적,층,년,월,건물나이,target_log_transformed,target/area_log_transformed,특수지구분명,공시지가(만원),표준지여부,지목코드,토지면적_log,용도지역코드1,용도지역코드2,지형높이코드,지형형상코드,도로접면코드,year_linear,month_sin,month_cos,x_norm,y_norm,x_2nd,y_2nd,x*y,x_sin,y_sin,x_cos,y_cos,대장구분명,(한강)오염행위 제한지역,(한강)폐기물매립시설 설치제한지역,가로구역별 최고높이 제한지역,가스공급설비,가축사육제한구역,개발제한구역,개발진흥지구,개발행위허가제한지역,건축선,건축용도지역기타,건축용도지역미분류,건축허가·착공제한지역,경관광장,경관녹지,경관지구,경관지구기타,고도지구,고등학교,고속철도,공공공지,공공도서관,공공문화체육시설미분류,공공시설구역,공공주택지구,공공지원민간임대주택 공급촉진지구,공공청사,공동구,공원,공원마을지구(공원집단시설지구),공원자연보존지구,공원자연환경지구,공익용산지,공익임지,공장설립승인지역,공장설립제한지역,공항소음피해예상지역,과밀억제권역,광로1류(폭 70M 이상),광로2류(폭 50M~70M),광로3류(폭 40M~50M),광역계획구역,광장,교육환경보호구역,교통광장,국가산업단지,국가지정문화재구역,국가하천,국립공원,국민임대주택단지예정지구,국지도로,국토이용기타용도지구,국토이용용도지구기타,근린공원,근린광장,근린상업지역,기타공공공지시설,기타공공청사시설,기타공원시설,기타교통시설,기타녹지시설,기타도로시설,기타도시공간시설,기타문화시설,기타보건위생시설,기타수도시설,기타시장시설,기타용도지역지구기타,기타용지,기타유통및공급시설,기타유통업무설비,기타자동차정류장,기타주차장시설,기타철도시설,기타폐기물처리시설,노외주차장,녹지,대공방어협조구역,대공방어협조구역(위탁고도:54-236m),대공방어협조구역(위탁고도:77-257m),대로1류(폭 35M~40M),대로2류(폭 30M~35M),대로3류(폭 25M~30M),대학,도로,도로구역,도서관,도시개발구역,도시개발구역기타,도시고속도로,도시관리계획 입안중,도시기타용도지역지구기타,도시기타용도지역지구미분류,도시기타용도지역지구용도지역지구,도시자연공원,도시자연공원구역,도시지역,도시지역기타,도시철도,묘지공원,문화공원,문화시설,문화재,문화재보존영향 검토대상구역,문화재보호구역,문화재보호구역기타,문화지구,박물관,방송통신시설,방수설비,방화지구,변전소(전원개발사업구역),변전시설,보전산지,보전임지,보존지구,보행자전용도로,부설주차장,비오톱1등급,비행안전제2구역(전술),비행안전제2구역(지원),비행안전제3구역(전술),비행안전제3구역(지원),비행안전제4구역(지원),비행안전제5구역(전술),비행안전제5구역(지원),비행안전제6구역(전술),사고지,사방시설,사방지,사업지역기타,사회복지시설,산업개발진흥지구,산업시설구역,상대보호구역,상수원보호구역,생산녹지지역,소공원,소로1류(폭 10M~12M),소로2류(폭 8M~10M),소로3류(폭 8M 미만),소하천,소하천구역,수도공급시설,수평표면구역,시·도 생태·경관보전지역,시가지경관지구,시가지조성사업지역,시도지정문화재구역,시장,시장정비구역,시장정비구역기타,아파트지구,야생생물보호구역,어린이공원,여객자동차터미널,역사도심,역사문화미관지구,역사문화특화경관지구,역사문화환경보존지역,연결녹지,연구개발특구,연구시설,열공급설비,온천공보호구역,완충녹지,용도구역기타,용도구역미분류,운동장,원추표면구역,원형보존지,유수지,유치원,유통단지,유통상업지역,유통업무설비,일단의공업용지조성사업지역,일단의주택단지조성사업지역,일반도로,일반미관지구,일반산업단지,일반상업지역,일반주거지역,일반철도,임업용산지,자동차검사시설,자동차전용도로,자동차정류장,자연경관지구,자연녹지지역,자연재해위험지구,장애물제한표면구역,재개발구역,재개발구역기타,재정비촉진지구,재정비촉진지구기타,저류시설,전기공급설비,전용주거지역,전이표면구역,전통사찰보존구역,절대보호구역,정비구역,정비구역기타,정비예정구역,제1종일반주거지역,제1종전용주거지역,제1종지구단위계획구역,제2종일반주거지역,제2종전용주거지역,제3종 구역,제3종일반주거지역,제방,제한보호구역,제한보호구역(전술항공:5km),제한보호구역(후방지역:500m),조례로정한지역,조망가로미관지구,조망가로특화경관지구,종합의료시설,주간선도로,주거용지,주거환경개선지구,주차장,주차장기타,주차환경개선지구,준공업지역,준보전산지,준주거지역,중로1류(폭 20M~25M),중로2류(폭 15M~20M),중로3류(폭 12M~15M),중심대광장,중심상업지역,중요시설물보호지구,중요시설물보호지구(공용),중요시설물보호지구(공항),중점경관관리구역,중학교,지구단위계획구역,지역특화발전특구,지원시설구역,지정문화재구역,지하광장,지하도로,진입표면구역,집단취락지구,철도,청사,청소년수련시설,체육공원,체육시설,초등학교,침수위험지구,택지개발예정지구,택지개발예정지구기타,토지거래계약에관한허가구역,토지구획정리사업지구기타,통제보호구역(민통선이남:300m),특수도로(보행자전용도로),특정개발진흥지구,특화경관지구,폐기물처리및재활용시설,하수도,하수종말처리시설,하천,하천구역,학교,학교이적지,현상변경허가 대상구역,홍수관리구역
0,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,733.118971,2,2013,9,25,10.950807,6.597308,1,921.0,0,8,7.063305,15,0,2,3,3,7.666667,-0.866025,-0.500000,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138,토지대장,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,0,77.75,57000,733.118971,2,2013,12,25,10.950807,6.597308,1,921.0,0,8,7.063305,15,0,2,3,3,7.916667,-0.500000,0.866025,5540.076032,-9058.615505,3.069244e+07,8.205851e+07,-5.018542e+07,-0.992492,0.986102,-0.122306,-0.166138,토지대장,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [64]:
merge5_df['대장구분명'].value_counts()

토지대장    3550554
임야대장         16
Name: 대장구분명, dtype: int64

In [65]:
merge5_df = merge5_df[merge5_df['대장구분명'] == '토지대장']
merge5_df.drop(columns=['대장구분명'], inplace=True)

In [66]:
merge5_df.drop(columns=['year_linear'], inplace=True)

In [68]:
land_plans_df.columns[3:]

Index(['(한강)오염행위 제한지역', '(한강)폐기물매립시설 설치제한지역', '가로구역별 최고높이 제한지역', '가스공급설비',
       '가축사육제한구역', '개발제한구역', '개발진흥지구', '개발행위허가제한지역', '건축선', '건축용도지역기타',
       ...
       '특화경관지구', '폐기물처리및재활용시설', '하수도', '하수종말처리시설', '하천', '하천구역', '학교', '학교이적지',
       '현상변경허가 대상구역', '홍수관리구역'],
      dtype='object', length=270)

In [72]:
cols_to_drop = []
for col in land_plans_df.columns[3:]:
    if (merge5_df[col].value_counts()[0] / merge5_df.shape[0]) > 0.999:
        cols_to_drop.append(col)
        
len(cols_to_drop)

108

In [73]:
merge5_df.drop(columns=cols_to_drop, inplace=True)

In [74]:
merge5_df.shape

(3550554, 195)

In [75]:
merge5_df.to_csv('training_data_ver_6.csv', index=False)